In [1]:
import numpy as np
import joblib
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [2]:
# Load all the saved models
random_forest = joblib.load(
    "../../../Models/requirement_relevancy_experiment/classifier_models/distilbert_random_forest_classifier.joblib"
)

gradient_boosting = joblib.load(
    "../../../Models/requirement_relevancy_experiment/classifier_models/distilbert_gradient_boost_classifier.joblib"
)

ada_boost = joblib.load(
    "../../../Models/requirement_relevancy_experiment/classifier_models/distilbert_adaboost_classifier.joblib"
)

xgboost = joblib.load(
    "../../../Models/requirement_relevancy_experiment/classifier_models/distilbert_xgboost_classifier.joblib"
)

In [3]:
# Load the resampled data

X_resampled = np.loadtxt(
    "../../../Datasets/irrelevant_requirements_dataset/distilbert_X_resampled.csv",
    delimiter=",",
)

y_resampled = np.loadtxt(
      "../../../Datasets/irrelevant_requirements_dataset/distilbert_y_resampled.csv",
    delimiter=",",
)

In [4]:
X_resampled.shape, y_resampled.shape

((1114, 49152), (1114,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42
)

In [6]:
validation_fold = 5

In [7]:
cross_validation_result_dict = {
    "model_name": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1": [],
    "fit_time": [],
    "score_time": [],
}


def append_scores(
    cross_validation_result_dict,
    model_name: str,
    accuracy: float,
    precision: float,
    recall: float,
    f1: float,
    fit_time: float,
    score_time: float,
):
    cross_validation_result_dict["model_name"].append(model_name)
    cross_validation_result_dict["accuracy"].append(accuracy)
    cross_validation_result_dict["precision"].append(precision)
    cross_validation_result_dict["recall"].append(recall)
    cross_validation_result_dict["f1"].append(f1)
    cross_validation_result_dict["fit_time"].append(fit_time)
    cross_validation_result_dict["score_time"].append(score_time)

In [8]:
diagonal_line = pd.DataFrame(
    {"False Positive Rate": [0, 1], "True Positive Rate": [0, 1]}
)


def show_roc_curve(model, model_name: str, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    true_positive_rate, false_positive_rate, _ = roc_curve(y_test, y_pred_proba)
    area_under_curve = roc_auc_score(y_test, y_pred_proba)

    fig = px.line(
        diagonal_line,
        x="False Positive Rate",
        y="True Positive Rate",
        title="ROC Curve",
    )

    fig.update_layout(
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        font=dict(size=16),
        legend=dict(
            bordercolor="Black",
            borderwidth=2,
            font=dict(size=18),
        ),
    )

    fig.update_xaxes(showline=True, linewidth=2, linecolor="black", mirror=True)

    fig.update_yaxes(showline=True, linewidth=2, linecolor="black", mirror=True)

    fig.add_trace(
        go.Scatter(
            x=true_positive_rate,
            y=false_positive_rate,
            mode="lines",
            name=f"{model_name} (AUC = {area_under_curve:.4f})",
        )
    )

    fig.show()

## Cross Validation

Cross validation is a technique for assessing how the results of a statistical analysis will generalize to an independent data set. It is mainly used in settings where the goal is prediction, and one wants to estimate how accurately a predictive model will perform in practice. One round of cross-validation involves partitioning a sample of data into complementary subsets, performing the analysis on one subset (called the training set), and validating the analysis on the other subset (called the validation set or testing set). To reduce variability, multiple rounds of cross-validation are performed using different partitions, and the validation results are averaged over the rounds.

### Cross Validation for Random Forest

In [9]:
random_forest_scores = cross_validate(
    random_forest,
    X_train,
    y_train,
    cv=validation_fold,
    n_jobs=-1,
    scoring=["f1", "accuracy", "precision", "recall"],
)

In [10]:
random_forest_scores

{'fit_time': array([22.20533156, 22.42752194, 22.47626781, 21.49058914, 22.16443825]),
 'score_time': array([0.04166436, 0.03743267, 0.03896499, 0.03983665, 0.04189897]),
 'test_f1': array([0.98850575, 0.99428571, 0.99428571, 0.97701149, 0.97206704]),
 'test_accuracy': array([0.98882682, 0.99438202, 0.99438202, 0.97752809, 0.97191011]),
 'test_precision': array([1.        , 0.98863636, 0.98863636, 0.97701149, 0.94565217]),
 'test_recall': array([0.97727273, 1.        , 1.        , 0.97701149, 1.        ])}

In [11]:
append_scores(
    cross_validation_result_dict,
    "Random Forest",
    np.mean(random_forest_scores["test_accuracy"]),
    np.mean(random_forest_scores["test_precision"]),
    np.mean(random_forest_scores["test_recall"]),
    np.mean(random_forest_scores["test_f1"]),
    np.mean(random_forest_scores["fit_time"]),
    np.mean(random_forest_scores["score_time"]),
)

In [12]:
show_roc_curve(random_forest, "Random Forest", X_test, y_test)

### Adaboost Cross Validation

In [13]:
adaboost_cross_validation_scores = cross_validate(
    ada_boost,
    X_train,
    y_train,
    cv=validation_fold,
    n_jobs=-1,
    scoring=["f1", "accuracy", "precision", "recall"],
)

adaboost_cross_validation_scores

{'fit_time': array([506.95300317, 517.9254024 , 510.22814775, 506.09332013,
        511.0671742 ]),
 'score_time': array([1.07113314, 0.86646819, 1.30598235, 1.09275723, 1.19018006]),
 'test_f1': array([0.93023256, 0.94674556, 0.93413174, 0.92682927, 0.92941176]),
 'test_accuracy': array([0.93296089, 0.9494382 , 0.93820225, 0.93258427, 0.93258427]),
 'test_precision': array([0.95238095, 0.97560976, 0.975     , 0.98701299, 0.95180723]),
 'test_recall': array([0.90909091, 0.91954023, 0.89655172, 0.87356322, 0.90804598])}

In [14]:
append_scores(
    cross_validation_result_dict,
    "AdaBoost",
    np.mean(adaboost_cross_validation_scores["test_accuracy"]),
    np.mean(adaboost_cross_validation_scores["test_precision"]),
    np.mean(adaboost_cross_validation_scores["test_recall"]),
    np.mean(adaboost_cross_validation_scores["test_f1"]),
    np.mean(adaboost_cross_validation_scores["fit_time"]),
    np.mean(adaboost_cross_validation_scores["score_time"]),
)

In [15]:
show_roc_curve(ada_boost, "AdaBoost", X_test, y_test)

### XGBoost Cross Validation

In [16]:
xgboost_scores = cross_validate(
    xgboost,
    X_train,
    y_train,
    cv=validation_fold,
    scoring=["f1", "accuracy", "precision", "recall"],
)

xgboost_scores

{'fit_time': array([ 86.6879189 , 106.56840658, 103.77299523,  90.91147685,
        100.04932857]),
 'score_time': array([0.19104028, 0.18222332, 0.1258378 , 0.13463712, 0.11307049]),
 'test_f1': array([0.95906433, 0.98285714, 1.        , 0.95857988, 0.96629213]),
 'test_accuracy': array([0.96089385, 0.98314607, 1.        , 0.96067416, 0.96629213]),
 'test_precision': array([0.98795181, 0.97727273, 1.        , 0.98780488, 0.94505495]),
 'test_recall': array([0.93181818, 0.98850575, 1.        , 0.93103448, 0.98850575])}

In [17]:
append_scores(
    cross_validation_result_dict,
    "XGBoost",
    np.mean(xgboost_scores["test_accuracy"]),
    np.mean(xgboost_scores["test_precision"]),
    np.mean(xgboost_scores["test_recall"]),
    np.mean(xgboost_scores["test_f1"]),
    np.mean(xgboost_scores["fit_time"]),
    np.mean(xgboost_scores["score_time"]),
)

In [18]:
show_roc_curve(xgboost, "XGBoost", X_test, y_test)

### Gradient Boosting Cross Validation


In [19]:
gradient_boosting_cross_validation_scores = cross_validate(
    gradient_boosting,
    X_train,
    y_train,
    cv=validation_fold,
    n_jobs=-1,
    scoring=["f1", "accuracy", "precision", "recall"],
)

In [20]:
append_scores(
    cross_validation_result_dict,
    "Gradient Boosting",
    np.mean(gradient_boosting_cross_validation_scores["test_accuracy"]),
    np.mean(gradient_boosting_cross_validation_scores["test_precision"]),
    np.mean(gradient_boosting_cross_validation_scores["test_recall"]),
    np.mean(gradient_boosting_cross_validation_scores["test_f1"]),
    np.mean(gradient_boosting_cross_validation_scores["fit_time"]),
    np.mean(gradient_boosting_cross_validation_scores["score_time"]),
)

In [21]:
show_roc_curve(xgboost, "XGBoost", X_test, y_test)

## Result Export

In [22]:
cross_validation_result_dict

{'model_name': ['Random Forest', 'AdaBoost', 'XGBoost', 'Gradient Boosting'],
 'accuracy': [0.9854058125666938,
  0.9371539765237588,
  0.9742012428598329,
  0.9786830707425773],
 'precision': [0.9799872790877288,
  0.9683621848814326,
  0.9796168715210737,
  0.9864045773348099],
 'recall': [0.9908568443051202,
  0.9013584117032393,
  0.9679728317659352,
  0.9702455590386625],
 'f1': [0.9852311418113768,
  0.9334701779590446,
  0.9733586973661577,
  0.9780300985964798],
 'fit_time': [22.152829742431642,
  510.4534095287323,
  97.59802522659302,
  2807.5523646354677],
 'score_time': [0.03995952606201172,
  1.105304193496704,
  0.14936180114746095,
  0.031714677810668945]}

In [23]:
from pandas import DataFrame

cross_validation_result_df = DataFrame(cross_validation_result_dict)
cross_validation_result_df.to_csv(
    "../../../Results/irrelevant_requirements_experiment/classifiers_with_distilbert_cross_validation_results.csv",
    index=False,
)

In [24]:
cross_validation_result_df

,model_name,accuracy,precision,recall,f1,fit_time,score_time
0,Random Forest,0.985406,0.979987,0.990857,0.985231,22.152830,0.039960
1,AdaBoost,0.937154,0.968362,0.901358,0.933470,510.453410,1.105304
2,XGBoost,0.974201,0.979617,0.967973,0.973359,97.598025,0.149362
3,Gradient Boosting,0.978683,0.986405,0.970246,0.978030,2807.552365,0.031715
